In [1]:
#Grab the basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Establish source path
source = 'competition_data'

In [5]:
from glob import glob

for path in glob(os.path.join(source, '*.csv')):
    filename = path.split('\\')[-1]
    filename = filename.split('.')[0]
    print(filename)

NameError: name 'os' is not defined

In [6]:
#Use os notebook is being run on to determine path syntax
#Use glob to make a dictionary of dataframes
import os

data = {}

for path in glob(os.path.join(source, '*.csv')):
    df = pd.read_csv(path)
    filename = path.split('\\')[-1]
    name = filename.split('.')[0]
    data[name] = df 

#Check our work
data.keys()

dict_keys(['bill_of_materials', 'components', 'comp_adaptor', 'comp_boss', 'comp_elbow', 'comp_float', 'comp_hfl', 'comp_nut', 'comp_other', 'comp_sleeve', 'comp_straight', 'comp_tee', 'comp_threaded', 'specs', 'test_set', 'train_set', 'tube', 'tube_end_form', 'type_component', 'type_connection', 'type_end_form'])

In [7]:
#Grab a few specific frames to start
materials = data['bill_of_materials']
components = data['components']
tube = data['tube']
trainval = data['train_set']
test = data['test_set']

In [8]:
components.describe()

,component_id,name,component_type_id
count,2048,2047,2048
unique,2048,297,29
top,C-0277,FLANGE,OTHER
freq,1,350,1006


In [9]:
#Get a tidy table of tube assembly components
assembly_components = materials.melt(id_vars = 'tube_assembly_id',
                                     value_vars = [f'component_id_{n}' for n in range(1,9)])
#Sort it by the tube assembly id, drop nas, and rename value to component id
assembly_components = (assembly_components
                       .sort_values(by='tube_assembly_id')
                       .dropna()
                       .rename(columns = {'value':'component_id'}))
#Have a look
assembly_components.head()

,tube_assembly_id,variable,component_id
0,TA-00001,component_id_1,C-1622
21198,TA-00001,component_id_2,C-1629
1,TA-00002,component_id_1,C-1312
2,TA-00003,component_id_1,C-1312
3,TA-00004,component_id_1,C-1312


In [10]:
#Merge in component types to add information
assembly_component_types = assembly_components.merge(components, how = 'left', )
assembly_component_types.head()

,tube_assembly_id,variable,component_id,name,component_type_id
0,TA-00001,component_id_1,C-1622,NUT-SWIVEL,CP-025
1,TA-00001,component_id_2,C-1629,SLEEVE-ORFS,CP-024
2,TA-00002,component_id_1,C-1312,NUT-FLARED,CP-028
3,TA-00003,component_id_1,C-1312,NUT-FLARED,CP-028
4,TA-00004,component_id_1,C-1312,NUT-FLARED,CP-028


In [21]:
#Make a crosstab out of assembly_component_types
table = pd.crosstab(assembly_component_types['tube_assembly_id'],
                    assembly_component_types['component_type_id'])
table = table.reset_index()
table.columns.name = ''
print(table.shape)
table.head()

(19149, 30)


,tube_assembly_id,CP-001,CP-002,CP-003,CP-004,CP-005,CP-006,CP-007,CP-008,CP-009,...,CP-021,CP-022,CP-023,CP-024,CP-025,CP-026,CP-027,CP-028,CP-029,OTHER
0,TA-00001,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,TA-00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,TA-00003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,TA-00004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,TA-00005,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [23]:
#Find components that share 'weight' feature
#Create a search function for later use

def search_column(name):
    for path in glob(os.path.join(source, '*.csv')):
        df = pd.read_csv(path)
        if name in df.columns:
            print(path,df.shape)
            print(df.columns.tolist(), '\n')
            
search_column('weight')

competition_data\comp_adaptor.csv (25, 20)
['component_id', 'component_type_id', 'adaptor_angle', 'overall_length', 'end_form_id_1', 'connection_type_id_1', 'length_1', 'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2', 'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2', 'nominal_size_2', 'hex_size', 'unique_feature', 'orientation', 'weight'] 

competition_data\comp_boss.csv (147, 15)
['component_id', 'component_type_id', 'type', 'connection_type_id', 'outside_shape', 'base_type', 'height_over_tube', 'bolt_pattern_long', 'bolt_pattern_wide', 'groove', 'base_diameter', 'shoulder_diameter', 'unique_feature', 'orientation', 'weight'] 

competition_data\comp_elbow.csv (178, 16)
['component_id', 'component_type_id', 'bolt_pattern_long', 'bolt_pattern_wide', 'extension_length', 'overall_length', 'thickness', 'drop_length', 'elbow_angle', 'mj_class_code', 'mj_plug_class_code', 'plug_diameter', 'groove', 'unique_feature', 'orientation', 'weight'] 

competi

In [24]:
#Many of these also have orientation and unique_feature features
comp_threaded = data['comp_threaded']
comp_threaded['unique_feature'].value_counts()

No     161
Yes     33
Name: unique_feature, dtype: int64

In [25]:
#Clearly these are binary, but in yes/no format. We'll note that for cleaning later
comp_threaded['orientation'].value_counts()

No     121
Yes     73
Name: orientation, dtype: int64

In [35]:
#Read all component dataframes and concatenate them into one df
df_list = [pd.read_csv(path) for path in glob(os.path.join(source, 'comp_*.csv'))]
comp = pd.concat(df_list, sort=False)
#Focus in on the shared features
columns = ['component_id', 'component_type_id', 'weight', 'unique_feature', 'orientation']
comp = comp[columns]
#Convert binarys to numeric values
comp['unique_feature'] = (comp['unique_feature']=='Yes').astype(int)
comp['orientation'] = (comp['orientation']=='Yes').astype(int)
#Fill weight nas with median
comp['weight'] = comp['weight'].fillna(comp['weight'].median())
del df_list

In [36]:
comp.head()

,component_id,component_type_id,weight,unique_feature,orientation
0,C-0005,CP-028,0.206,0,0
1,C-0006,CP-028,0.083,0,0
2,C-1435,CP-028,0.023,0,0
3,C-1546,CP-028,0.026,0,0
4,C-1583,CP-028,0.256,0,0


In [ ]:
#Create features for total components and number of distinct components
materials['components_total'] = sum(materials[f'quantity_{n}'.fillna(0) for n in range(1,9)])
materials['components_distinct'] = sum(materials[f'component_id_{n}'
                                                 .notnull()
                                                 .astype(int) for n in range(1,9)])
#Create matching columns in materials
materials['orientation'] = 0
materials['unique_feature'] = 0
materials['weight'] = 0

for n in range(1,9):
    materials = materials.merge(comp, left_on = f'component_id_{n}',
                                right_on = 'component_id', how = 'left',
                                suffixes = ('', f'_{n}'))
    
for col in materials:
    if 'orientation' in col or 'weight' in col or 'unique_feature' in col:
        materials[col] = materials[col].fillna(0)
        
materials['orientation'] = sum(materials[f'orientation_{n}'] for n in range(1,9))